# memo

# import など準備

## import, MySQL 接続

In [1]:
# import
import sys
import os
import re
import datetime as dt
import time
import importlib
import logging
import numpy as np
import pandas as pd
from robobrowser import RoboBrowser
# from robobrowser.browser import RoboState
from retry import retry
from dateutil.parser import parse
from datetime import datetime

import stock

In [ ]:
importlib.reload(stock)

In [9]:
# pandas の最大表示列数を設定 (max_rows で表示行数の設定も可能)
pd.set_option('display.max_columns', 30)

In [5]:
sql = stock.sql()

## 比較参照用、株プロ決算の読み込み

In [3]:
table_name = 'kabupro_kessan'

In [6]:
kabupro = sql.read_table(table_name)

In [10]:
kabupro

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目 (売上高欄),売上高,営業利益,経常利益,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開日 (更新日)
0,1301,極洋,日本基準,個別,2013年3月期,第2四半期,2012-04-01,2012-09-30,売上高,7.654600e+10,1.530000e+08,3.530000e+08,7.900000e+07,0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-10-21
1,1301,極洋,日本基準,個別,2013年3月期,通期,2012-04-01,2013-03-31,売上高,1.606100e+11,9.900000e+08,1.392000e+09,8.060000e+08,7.68,NaN,1.481800e+10,7.049200e+10,141.08,NaN,NaN,NaN,2014-05-09
2,1301,極洋,日本基準,個別,2014年3月期,第2四半期,2013-04-01,2013-09-30,売上高,8.816100e+10,1.367000e+09,1.457000e+09,1.834000e+09,17.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-10-27
3,1301,極洋,日本基準,個別,2014年3月期,通期,2013-04-01,2014-03-31,売上高,1.920260e+11,2.471000e+09,2.546000e+09,1.806000e+09,17.20,16.77,1.616400e+10,7.038100e+10,153.90,NaN,NaN,NaN,2015-05-08
4,1301,極洋,日本基準,個別,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,9.947600e+10,8.900000e+08,9.500000e+08,2.504000e+09,23.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
5,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,1.894000e+09,2.042000e+09,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2015-05-08
6,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,1.894000e+09,2.042000e+09,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2016-05-09
7,1301,極洋,日本基準,個別,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.163500e+11,2.211000e+09,2.432000e+09,1.259000e+09,119.87,110.51,1.781700e+10,7.814000e+10,1696.42,NaN,NaN,NaN,2017-05-11
8,1301,極洋,日本基準,個別,2017年3月期,通期,2016-04-01,2017-03-31,売上高,2.280830e+11,3.187000e+09,2.835000e+09,2.194000e+09,208.94,192.95,2.011700e+10,8.130600e+10,1915.42,NaN,NaN,NaN,2017-05-11
9,1301,極洋,日本基準,連結,2013年3月期,第1四半期,2012-04-01,2012-06-30,売上高,4.319100e+10,4.690000e+08,6.770000e+08,3.350000e+08,3.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-05


In [5]:
kabupro.columns

Index(['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目 (売上高欄)', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益',
       '希薄化後一株当り純利益', '純資産又は株主資本', '総資産', '一株当り純資産', '営業キャッシュフロー',
       '投資キャッシュフロー', '財務キャッシュフロー', '情報公開日 (更新日)'],
      dtype='object')

In [43]:
kabupro.ix[14]
# 株プロにしか無い項目: 希薄化後一株当り純利益, 純資産又は株主資本, 営業キャッシュフロー, 投資キャッシュフロー, 財務キャッシュフロー

証券コード                            1301
企業名                                極洋
会計基準                             日本基準
連結個別                               連結
決算期                          2014年3月期
決算期間                            第2四半期
期首                2013-04-01 00:00:00
期末                2013-09-30 00:00:00
名寄前勘定科目 (売上高欄)                    売上高
売上高                        9.4101e+10
営業利益                        1.404e+09
経常利益                          1.5e+09
純利益                          1.88e+09
一株当り純利益                          17.9
希薄化後一株当り純利益                       NaN
純資産又は株主資本                         NaN
総資産                               NaN
一株当り純資産                           NaN
営業キャッシュフロー                        NaN
投資キャッシュフロー                        NaN
財務キャッシュフロー                        NaN
情報公開日 (更新日)       2014-11-07 00:00:00
Name: 14, dtype: object

## 参考コード

In [ ]:
def get_price_yahoojp(code, start=None, end=None, interval='d'): # start = '2017-01-01'
    # http://sinhrks.hatenablog.com/entry/2015/02/04/002258
    # http://jbclub.xii.jp/?p=598
    base = 'http://info.finance.yahoo.co.jp/history/?code={0}.T&{1}&{2}&tm={3}&p={4}'
    
    start = pd.to_datetime(start) # Timestamp('2017-01-01 00:00:00')

    if end == None:
        end = pd.to_datetime(pd.datetime.now())
    else :
        end = pd.to_datetime(end)
    start = 'sy={0}&sm={1}&sd={2}'.format(start.year, start.month, start.day) # 'sy=2017&sm=1&sd=1'
    end = 'ey={0}&em={1}&ed={2}'.format(end.year, end.month, end.day)
    p = 1
    tmp_result = []

    if interval not in ['d', 'w', 'm', 'v']:
        raise ValueError("Invalid interval: valid values are 'd', 'w', 'm' and 'v'")

    while True:
        url = base.format(code, start, end, interval, p)
        # print(url)
        # https://info.finance.yahoo.co.jp/history/?code=7203.T&sy=2000&sm=1&sd=1&ey=2017&em=10&ed=13&tm=d&p=1
        tables = get_table(url)
        if len(tables) < 2 or len(tables[1]) == 0:
            # print('break')
            break
        tmp_result.append(tables[1]) # ページ内の3つのテーブルのうち2番目のテーブルを連結
        p += 1
        # print(p)
        
    result = pd.concat(tmp_result, ignore_index=True) # インデックスをゼロから振り直す

    result.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'AdjClose'] # 列名を変更
    if interval == 'm':
        result['Date'] = pd.to_datetime(result['Date'], format='%Y年%m月')
    else:
        result['Date'] = pd.to_datetime(result['Date'], format='%Y年%m月%d日') # 日付の表記を変更
    result = result.set_index('Date') # インデックスを日付に変更
    result = result.sort_index()
    
    stock_name = tables[0].columns[0]
    # print([code, stock_name])
    
    return [result, stock_name]

In [ ]:
# yahoo 初回連続読み込み
# 読み込み期間の設定
start = '2000-01-01'
end = None

# ロガー設定
start_time = dt.datetime.now()
logging.basicConfig(filename='get_price_{0}.log'.format(start_time.strftime('%Y-%m-%d')), filemode='w', level=logging.INFO)
logging.info('{0} get_price Started'.format(start_time.strftime('%Y-%m-%d %H:%M:%S')))

sql = stock.sql() # MySQLに接続するクラスインスタンスを作成

info = sql.get_info() # 保存済み info の読み込み
failed = [] # 読み込みに失敗した銘柄のコードを書き込むリストを作成
save_failed = [] # 保存のみ失敗した分

# 連続読み込み書き込み
for index in range(len(reading_code)):
    code = reading_code[index]
    
    try:
        time.sleep(5)
        
        # Yahooファイナスンスから時系列情報と銘柄名を取得
        tmp_price, stock_name = stock.get_price_yahoojp(code, start=start, end=end)
        
        # 価格と価格以外の情報を分離
        tmp_info = tmp_price[tmp_price.isnull().any(axis=1)].reset_index()
        if len(tmp_info) > 0:
            new_info = stock.reform_info(tmp_info, code, stock_name)
            info = info.append(new_info, ignore_index=True)
        
            price = stock.extract_price(tmp_price)
            
        else:
            price = tmp_price # 価格以外の情報がなければそのまま
            
        try:
            # CSVで保存
            price.to_csv('/Users/Really/Stockyard/_csv/t_{0}.csv'.format(code))
            info.to_csv('/Users/Really/Stockyard/_csv/info.csv')
            # MySQLに保存
            sql.write_price(code, price)
            sql.write_info('info', info)
          
            print('{0}: Success {1}'.format(index, code))
            
        except Exception as e:
            logging.warning('{0} {1}: {2}'.format(dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), code, e))
            save_failed.append(code)
            print('{0}: Failed in {1} at Save Data'.format(index, code))
            print(e)
            
    except Exception as e:
        logging.warning('{0} {1}: {2}'.format(dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), code, e))
        failed.append(code)
        print('{0}: Failed in {1} at get_price'.format(index, code))
        print(e)

print('Failed in {0} stocks at get:'.format(len(failed)))
print(failed)
print('Failed in {0} stocks at save:'.format(len(save_failed)))
print(save_failed)

# 最後にinfoの重複と順序を整理してから再度保存
info = info.drop_duplicates()
info = info.sort_values(by=['Code', 'Date'])
info.to_csv('/Users/Really/Stockyard/_csv/info.csv')
sql.write_info('info', info)

logging.info('{0} get_price Finished'.format(dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')))

In [ ]:
%%writefile amazon_order_history.py

# Amazon.co.jpの注文履歴を取得する

import sys
import os
from robobrowser import RoboBrowser

# 認証の情報は環境変数から取得する
AMAZON_EMAIL = os.environ['AMAZON_EMAIL']
AMAZON_PASSWORD = os.environ['AMAZON_PASSWORD']

# RoboBrowserオブジェクトを作成する
browser = RoboBrowser(
    parser='html.parser', # Beatiful Soupで使用するパーサーを指定
    # Cookieが使用できないと表示されてログインできない問題を回避するため
    # 通常のブラウザーのUser-Agent(ここではFirefoxのもの)を使う
    user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:45.0) Gecko/20100101 Firefox/45.0')


def main():
    # 注文履歴のページを開く
    print('Navigating...', file=sys.stderr)
    browser.open('https://www.amazon.co.jp/gp/css/order-history')
    
    # サインインページにリダイレクトされていることを確認する
    assert 'Amazonサインイン' in browser.parsed.title.string
    
    # name="signIn" というサインインフォームを埋める。
    # フォームのname属性の値はブラウザーの開発者ツールで確認できる。
    form = browser.get_form(attrs={'name': 'signIn'})
    form['email'] = AMAZON_EMAIL
    form['password'] = AMAZON_PASSWORD
    
    # フォームを送信する。正常にログインするにはRefererヘッダーとAccept-Languageヘッダーが必要。
    print('Signing in...', file=sys.stderr)
    browser.submit_form(form, headers={
        'Referer': browser.url,
        'Accept-Language': 'ja,en-US;q=0.7,en;q=0.3',
    })
    
    # ログインに失敗する場合は、次の行のコメントを外してHTMLのソースを確認すると良い。
    # print(browser.parsed.prettify())

    # ページャーをたどる。
    while True:
        assert '注文履歴' in browser.parsed.title.string # 注文履歴画面が表示されていることを確認する。
        
        print_order_history() # 注文履歴を表示する。
        
        link_to_next = browser.get_link('次へ') #「次へ」というテキストを持つリンクを取得する。
        if not link_to_next:
            break #「次へ」のリンクがない場合はループを抜けて終了する。
            
        print('Following link to next page...', file=sys.stderr)
        browser.follow_link(link_to_next) # 次へ」というリンクをたどる。
        
        
def print_order_history():
    """
    現在のページのすべての注文履歴を表示する
    """
    for line_item in browser.select('.order-info'):
        order = {} # 注文の情報を格納するためのdict
        # ページ内のすべての注文履歴について反復する。ブラウザーの開発者ツールでclass属性の値を確認できる
        # 注文の情報のすべての列について反復する
        for column in line_item.select('.a-column'):
            label_element = column.select_one('.label')
            value_element = column.select_one('.value')
            # ラベルと値がない列は無視する。
            if label_element and value_element:
                label = label_element.get_text().strip()
                value = value_element.get_text().strip()
                order[label] = value
        print(order['注文日'], order['合計']) # 注文の情報を表示する。
        

if __name__ == '__main__':
    main()

In [ ]:
!forego run python amazon_order_history.py

# ログイン。入力したメアドとパスワードが出力に表示されてしまうのでそのままでGitHubに上げちゃダメ！ログインを済ませたら必ずすぐにクリア！

__TODO__ forego を利用してログイン情報を隠せないか検討

In [ ]:
# sign-in
# 認証の情報
KT_EMAIL = input('Name?')
KT_PASSWORD = input('Password?')

# RoboBrowserオブジェクトを作成する
browser = RoboBrowser(
    parser='html.parser', # Beatiful Soupで使用するパーサーを指定
    # Cookieが使用できないと表示されてログインできない問題を回避するため
    # 通常のブラウザーのUser-Agent(ここではFirefoxのもの)を使う
    user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:45.0) Gecko/20100101 Firefox/45.0')

# ログインページを開く
print('Navigating...', file=sys.stderr)
browser.open('https://account.kabutan.jp/login')

# 株探プレミアムページにいることを確認する
assert '株探プレミアム' in browser.parsed.title.string

# name="signIn" というサインインフォームを埋める。
# フォームのname属性の値はブラウザーの開発者ツールで確認できる。
form = browser.get_form(action='/login')
form['session[email]'] = KT_EMAIL
form['session[password]'] = KT_PASSWORD

# フォームを送信する。
# 正常にログインするにはRefererヘッダーとAccept-Languageヘッダーが必要な場合がある。
print('Signing in...', file=sys.stderr)
browser.submit_form(form)

print(browser.select('.is-success')[0].text.strip())

# 銘柄コードリスト

In [16]:
domestic_stock_table = sql.read_table('domestic_stock_table')

In [17]:
domestic_stock_table

,date,code,name,market,code_33,category_33,code_17,category_17,code_scale,scale
0,20171031,1301,極洋,市場第一部（内国株）,50,水産・農林業,1,食品,7,TOPIX Small 2
1,20171031,1332,日本水産,市場第一部（内国株）,50,水産・農林業,1,食品,4,TOPIX Mid400
2,20171031,1333,マルハニチロ,市場第一部（内国株）,50,水産・農林業,1,食品,4,TOPIX Mid400
3,20171031,1352,ホウスイ,市場第一部（内国株）,6050,卸売業,13,商社・卸売,7,TOPIX Small 2
4,20171031,1376,カネコ種苗,市場第一部（内国株）,50,水産・農林業,1,食品,7,TOPIX Small 2
5,20171031,1377,サカタのタネ,市場第一部（内国株）,50,水産・農林業,1,食品,6,TOPIX Small 1
6,20171031,1379,ホクト,市場第一部（内国株）,50,水産・農林業,1,食品,6,TOPIX Small 1
7,20171031,1380,秋川牧園,JASDAQ(スタンダード・内国株）,50,水産・農林業,1,食品,-,-
8,20171031,1381,アクシーズ,JASDAQ(スタンダード・内国株）,50,水産・農林業,1,食品,-,-
9,20171031,1382,ホーブ,JASDAQ(スタンダード・内国株）,50,水産・農林業,1,食品,-,-


In [18]:
code_list = list(domestic_stock_table['code'])

In [74]:
code_list[-10:]

[9987, 9989, 9990, 9991, 9992, 9993, 9994, 9995, 9996, 9997]

In [20]:
len(code_list)

3548

In [75]:
start_index = 30
increase_number = 10
# end_index = start_index + increase_number
end_index = len(code_list)

reading_code = code_list[start_index : end_index]
print(reading_code[-10:])
print('Next start from {0}'.format(start_index + increase_number))

[9987, 9989, 9990, 9991, 9992, 9993, 9994, 9995, 9996, 9997]
Next start from 40


# 上場日本株全銘柄の決算ページの連続読み込み

## get_html 関数 (retry 付き)

In [56]:
@retry(tries=5, delay=1, backoff=2)
def get_html(url):
    browser.open(url)
    assert '決算' in browser.parsed.title.string # 決算ページにいることを確認する
    stock_name = browser.select('.kobetsu_data_table1_meigara')[0].text.strip()
    print('{0}: {1}'.format(code, stock_name))
    result = browser.find()
    
    return result

## 連続読み込み

In [76]:
# ロガー設定
start_time = dt.datetime.now()
logging.basicConfig(filename='get_kabutan_html_{0}.log'.format(start_time.strftime('%Y-%m-%d')), filemode='w', level=logging.INFO)
logging.info('{0} get_html Started'.format(start_time.strftime('%Y-%m-%d %H:%M:%S')))

failed = [] # 読み込みに失敗した銘柄のコードを書き込むリストを作成


# 連続読み込み書き込み
for index in range(len(reading_code)):
    code = reading_code[index]
    
    try:
        time.sleep(3 + np.random.randint(0, 3))
        
        url = 'https://kabutan.jp/stock/finance?code={0}&mode=k'.format(code)
        result = get_html(url)

        with open('/Users/Really/Stockyard/_kabutan_html/kabutan_{0}.html'.format(code), 'w') as write_html:
            write_html.write(str(result))
                    
    except Exception as e:
        logging.warning('{0} {1}: {2}'.format(dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), code, e))
        failed.append(code)
        print('{0}: Failed in {1} at get_html'.format(index, code))
        print(e)


print('Failed in {0} stocks at get:'.format(len(failed)))
print(failed)

logging.info('{0} get_html Finished'.format(dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')))

1514: 住石ホールディングス
1515: 日鉄鉱業
1518: 三井松島産業
1605: 国際石油開発帝石
1606: 日本海洋掘削
1662: 石油資源開発
1663: Ｋ＆Ｏエナジーグループ
1711: 省電舎ホールディングス
1712: ダイセキＳ
1716: 第一カッター興業
1717: 明豊ファシリ
1718: 美樹工業
1719: 安藤・間
1720: 東急建設
1721: コムシスＨＤ
1722: ミサワホーム
1723: 日本電技
1724: シンクレイヤ
1726: Ｂｒ．ＨＤ
1728: ミサワホーム中国
1730: 麻生フオームクリート
1736: オーテック
1737: 三井金エンジ
1739: シード平和
1743: コーアツ工業
1757: クレアホールディングス
1758: 太洋基礎工業
1762: 高松グループ
1764: 工藤建設
1766: 東建コーポレーション
1768: ソネック
1770: 藤田エンジニアリング
1775: 富士古河Ｅ＆Ｃ
1776: 三井住建道路
1780: ヤマウラ
1782: 常磐開発
1783: アジアＧＨＤ
1787: ナカボーテック
1788: 三東工業社
1789: 山加電業
1793: 大本組
1795: マサル
1798: 守谷商会
1799: 第一建設工業
1801: 大成建設
1802: 大林組
1803: 清水建設
1805: 飛島建設
1807: 佐藤渡辺
1808: 長谷工コーポレーション
1810: 松井建設
1811: 錢高組
1812: 鹿島
1813: 不動テトラ
1814: 大末建設
1815: 鉄建
1820: 西松建設
1821: 三井住友建設
1822: 大豊建設
1824: 前田建設工業
1826: 佐田建設
1827: ナカノフドー建設
1828: 田辺工業
1833: 奥村組
1835: 東鉄工業
1840: 土屋ホールディングス
1841: サンユー建設
1844: 大盛工業
1846: 鈴縫工業
1847: イチケン
1848: 富士ピー・エス
1850: 南海辰村建設
1852: 淺沼組
1853: 森組
1860: 戸田建設
1861: 熊谷組
1865: 青木あすなろ建設
1866: 北野建設
1867: 植木組
1868: 三井ホーム
1870

## 確認

In [12]:
code = 7203

In [90]:
# 保存した html からテーブルを読み込んでみる
tables = pd.read_html('/Users/Really/Stockyard/_kabutan_html/kabutan_{0}.html'.format(code), header=0)

In [91]:
# 通期業績
tables[11]

,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,連 1999.03,12749008.0,774945.0,771885.0,356180.0,94.2,23,99/05/24
2,連 2000.03,12879561.0,775982.0,797058.0,406798.0,110.0,24,00/05/17
3,連 2001.03,13424423.0,870131.0,972273.0,471295.0,127.9,25,01/05/16
4,連 2002.03,15106297.0,1123470.0,1113524.0,615824.0,170.7,28,02/05/13
5,U 2003.03,15501553.0,1271646.0,1226652.0,750942.0,211.3,36,03/05/08
6,U 2004.03,17294760.0,1666890.0,1765793.0,1162098.0,342.9,45,04/05/11
7,U 2005.03,18551526.0,1672187.0,1754637.0,1171260.0,355.4,65,05/05/10
8,U 2006.03,21036909.0,1878342.0,2087360.0,1372180.0,421.8,90,06/05/10
9,U 2007.03,23948091.0,2238683.0,2382516.0,1644032.0,512.1,120,07/05/09


In [92]:
# 業績予想
tables[12]

,決算期,修正日,修正方向,売上高,営業益,経常益,最終益,修正配当,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,U,2013.03,12/05/09,NaN,NaN,NaN,NaN,NaN,NaN,22000000.0,1000000.0,1160000.0,760000.0,－
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,U,NaN,12/11/05,↓↑↑↑‐,↓,↑,↑,↑,‐,21300000.0,1050000.0,1180000.0,780000.0,－
3,↓,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,U,NaN,13/02/05,↑↑↑↑‐,↑,↑,↑,↑,‐,21800000.0,1150000.0,1290000.0,860000.0,－
5,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,U,実績,13/05/08,↑↑↑↑‐,↑,↑,↑,↑,‐,22064192.0,1320888.0,1403649.0,962163.0,90
7,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,U,2014.03,13/05/08,NaN,NaN,NaN,NaN,NaN,NaN,23500000.0,1800000.0,1890000.0,1370000.0,－
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 保存した html ファイルからテーブル属性のみ読み込み、整形

__TODO__

In [155]:
# 保存した html からテーブル属性を読み込み
tables = pd.read_html('/Users/Really/Stockyard/_kabutan_html/kabutan_{0}.html'.format(code), header=0)

## リスト、テーブルの概要

In [7]:
len(tables)

29

In [8]:
len(tables[12])

52

In [9]:
len(tables[12].columns)

14

## tables[7] 銘柄概要

In [8]:
tables[7]

,7203,トヨタ自動車,東証１,15:00


##  tables[9] 銘柄概要

In [11]:
tables[9]

,業績：
0,輸送用機器
1,単位 100株


## tables [10] ＰＥＲ ＰＢＲ 利回り 信用倍率 (データ取得日時点?)

In [12]:
tables[10]

,ＰＥＲ,ＰＢＲ,利回り,信用倍率
0,10.6倍,1.13倍,－％,1.53倍


## tables[11] 通期業績

In [155]:
# 保存した html からテーブル属性を読み込み
tables = pd.read_html('/Users/Really/Stockyard/_kabutan_html/kabutan_{0}.html'.format(code), header=0)

In [256]:
tables[11]
# 株プロに無い項目: １株配

,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,連 1999.03,12749008.0,774945.0,771885.0,356180.0,94.2,23,99/05/24
2,連 2000.03,12879561.0,775982.0,797058.0,406798.0,110.0,24,00/05/17
3,連 2001.03,13424423.0,870131.0,972273.0,471295.0,127.9,25,01/05/16
4,連 2002.03,15106297.0,1123470.0,1113524.0,615824.0,170.7,28,02/05/13
5,U 2003.03,15501553.0,1271646.0,1226652.0,750942.0,211.3,36,03/05/08
6,U 2004.03,17294760.0,1666890.0,1765793.0,1162098.0,342.9,45,04/05/11
7,U 2005.03,18551526.0,1672187.0,1754637.0,1171260.0,355.4,65,05/05/10
8,U 2006.03,21036909.0,1878342.0,2087360.0,1372180.0,421.8,90,06/05/10
9,U 2007.03,23948091.0,2238683.0,2382516.0,1644032.0,512.1,120,07/05/09


In [257]:
# 全ての列項目がnullの行を除去
tables[11] = tables[11][~tables[11].isnull().all(axis=1)].reset_index(drop=True)

In [258]:
# 予想値と前期比の行を除去
tables[11] = tables[11][~((tables[11]['決算期'].str.contains('予')) | (tables[11]['決算期'].str.contains('前期比')))].reset_index(drop=True)

In [259]:
# 決算期列の要素を会計基準と決算期に分割、それぞれの列に代入(同時に会計基準列を新規作成)
tables[11][['会計基準', '決算期']] = pd.DataFrame(list(tables[11]['決算期'].str.split(' ')))

In [260]:
# 列の並び替え
tables[11] = tables[11][['会計基準', '決算期', '売上高', '営業益', '経常益', '最終益', '１株益', '１株配', '発表日']]

In [261]:
# 100万円単位換算
tables[11][['売上高', '営業益', '経常益', '最終益']] = tables[11][['売上高', '営業益', '経常益', '最終益']].apply(lambda x: x * 1000000)

In [262]:
# 型変換
tables[11]['１株配'] = tables[11]['１株配'].astype(float)

In [263]:
# 日付のパース、datetime.dateへの型変換
# tables[11]['決算期'] = tables[11]['決算期'].apply(lambda x: datetime.strptime(x, '%Y.%m').date()) # 日付ではないので文字列のままの方がいいかも？
tables[11]['発表日'] = tables[11]['発表日'].apply(lambda x: parse(x, yearfirst=True).date())
# pandasのTimestampへの型変換
tables[11]['発表日'] = pd.to_datetime(tables[11]['発表日'], format='%Y-%m-%d')
# tables[11]['決算期'] = pd.to_datetime(tables[11]['決算期'], format='%Y-%m-%d')

In [264]:
tables[11].dtypes

会計基準            object
決算期             object
売上高            float64
営業益            float64
経常益            float64
最終益            float64
１株益            float64
１株配            float64
発表日     datetime64[ns]
dtype: object

In [296]:
tables[11]

,会計基準,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,連,1999.03,1.274901e+13,7.749450e+11,7.718850e+11,3.561800e+11,94.2,23.0,1999-05-24
1,連,2000.03,1.287956e+13,7.759820e+11,7.970580e+11,4.067980e+11,110.0,24.0,2000-05-17
2,連,2001.03,1.342442e+13,8.701310e+11,9.722730e+11,4.712950e+11,127.9,25.0,2001-05-16
3,連,2002.03,1.510630e+13,1.123470e+12,1.113524e+12,6.158240e+11,170.7,28.0,2002-05-13
4,U,2003.03,1.550155e+13,1.271646e+12,1.226652e+12,7.509420e+11,211.3,36.0,2003-05-08
5,U,2004.03,1.729476e+13,1.666890e+12,1.765793e+12,1.162098e+12,342.9,45.0,2004-05-11
6,U,2005.03,1.855153e+13,1.672187e+12,1.754637e+12,1.171260e+12,355.4,65.0,2005-05-10
7,U,2006.03,2.103691e+13,1.878342e+12,2.087360e+12,1.372180e+12,421.8,90.0,2006-05-10
8,U,2007.03,2.394809e+13,2.238683e+12,2.382516e+12,1.644032e+12,512.1,120.0,2007-05-09
9,U,2008.03,2.628924e+13,2.270375e+12,2.437222e+12,1.717879e+12,540.6,140.0,2008-05-08


In [20]:
kabupro.ix[(kabupro['証券コード'] == code) & (kabupro['会計基準'] == '米国基準') & (kabupro['決算期間'] == '通期'), 
           ['連結個別', '決算期', '期首', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益', '情報公開日 (更新日)']]

,連結個別,決算期,期首,売上高,営業利益,経常利益,純利益,一株当り純利益,情報公開日 (更新日)
70487,連結,2013年3月期,2012-04-01,2.206419e+13,1.320888e+12,1.403649e+12,9.621630e+11,303.82,2014-05-08
70491,連結,2014年3月期,2013-04-01,2.569191e+13,2.292112e+12,2.441080e+12,1.823119e+12,575.30,2015-05-08
70498,連結,2015年3月期,2014-04-01,2.723452e+13,2.750564e+12,2.892828e+12,2.173338e+12,688.02,2015-05-08
70499,連結,2015年3月期,2014-04-01,2.723452e+13,2.750564e+12,2.892828e+12,2.173338e+12,688.02,2016-05-11
70503,連結,2016年3月期,2015-04-01,2.840312e+13,2.853971e+12,2.983381e+12,2.312694e+12,741.36,2017-05-10
70507,連結,2017年3月期,2016-04-01,2.759719e+13,1.994372e+12,2.193825e+12,1.831109e+12,605.47,2017-05-10


In [42]:
kabupro.columns

Index(['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目 (売上高欄)', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益',
       '希薄化後一株当り純利益', '純資産又は株主資本', '総資産', '一株当り純資産', '営業キャッシュフロー',
       '投資キャッシュフロー', '財務キャッシュフロー', '情報公開日 (更新日)'],
      dtype='object')

## tables[12] 業績予想

In [316]:
# 保存した html からテーブル属性を読み込み
tables = pd.read_html('/Users/Really/Stockyard/_kabutan_html/kabutan_{0}.html'.format(code), header=0)

In [247]:
tables[12]

,会計基準,決算期,売上高,営業益,経常益,最終益,修正配当,修正日
0,U,2013.03,22000000.0,1000000.0,1160000.0,760000.0,－,12/05/09
1,U,NaN,21300000.0,1050000.0,1180000.0,780000.0,－,12/11/05
2,U,NaN,21800000.0,1150000.0,1290000.0,860000.0,－,13/02/05
3,U,実績,22064192.0,1320888.0,1403649.0,962163.0,90,13/05/08
4,U,2014.03,23500000.0,1800000.0,1890000.0,1370000.0,－,13/05/08
5,U,NaN,24000000.0,1940000.0,2030000.0,1480000.0,－,13/08/02
6,U,NaN,25000000.0,2200000.0,2290000.0,1670000.0,－,13/11/06
7,U,NaN,25500000.0,2400000.0,2530000.0,1900000.0,－,14/02/04
8,U,実績,25691911.0,2292112.0,2441080.0,1823119.0,165,14/05/08
9,U,2015.03,25700000.0,2300000.0,2390000.0,1780000.0,－,14/05/08


In [23]:
tables[12].columns

Index(['決算期', '修正日', '修正方向', '売上高', '営業益', '経常益', '最終益', '修正配当', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13'],
      dtype='object')

In [317]:
tables[12].columns = ['会計基準', '決算期', '発表日', 
                                   '結合修正方向', '売上高修正方向', '営業益修正方向', '経常益修正方向', '最終益修正方向', '修正配当修正方向', 
                                   '予想売上高', '予想営業益', '予想経常益', '予想最終益', '予想修正配当',]

In [25]:
# 実体行
tables[12][tables[12].index % 2 == 0].reset_index(drop=True)

,会計基準,決算期,修正日,結合修正方向,売上高修正方向,営業益修正方向,経常益修正方向,最終益修正方向,修正配当修正方向,売上高,営業益,経常益,最終益,修正配当
0,U,2013.03,12/05/09,NaN,NaN,NaN,NaN,NaN,NaN,22000000.0,1000000.0,1160000.0,760000.0,－
1,U,NaN,12/11/05,↓↑↑↑‐,↓,↑,↑,↑,‐,21300000.0,1050000.0,1180000.0,780000.0,－
2,U,NaN,13/02/05,↑↑↑↑‐,↑,↑,↑,↑,‐,21800000.0,1150000.0,1290000.0,860000.0,－
3,U,実績,13/05/08,↑↑↑↑‐,↑,↑,↑,↑,‐,22064192.0,1320888.0,1403649.0,962163.0,90
4,U,2014.03,13/05/08,NaN,NaN,NaN,NaN,NaN,NaN,23500000.0,1800000.0,1890000.0,1370000.0,－
5,U,NaN,13/08/02,↑↑↑↑‐,↑,↑,↑,↑,‐,24000000.0,1940000.0,2030000.0,1480000.0,－
6,U,NaN,13/11/06,↑↑↑↑‐,↑,↑,↑,↑,‐,25000000.0,2200000.0,2290000.0,1670000.0,－
7,U,NaN,14/02/04,↑↑↑↑‐,↑,↑,↑,↑,‐,25500000.0,2400000.0,2530000.0,1900000.0,－
8,U,実績,14/05/08,↑↓↓↓‐,↑,↓,↓,↓,‐,25691911.0,2292112.0,2441080.0,1823119.0,165
9,U,2015.03,14/05/08,NaN,NaN,NaN,NaN,NaN,NaN,25700000.0,2300000.0,2390000.0,1780000.0,－


In [31]:
# 不要行
tables[12][tables[12].index % 2 != 0]

,会計基準,決算期,修正日,結合修正方向,売上高修正方向,営業益修正方向,経常益修正方向,最終益修正方向,修正配当修正方向,売上高,営業益,経常益,最終益,修正配当
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,↓,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,↑,↓,↓,↓,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [318]:
tables[12].columns

Index(['会計基準', '決算期', '発表日', '結合修正方向', '売上高修正方向', '営業益修正方向', '経常益修正方向',
       '最終益修正方向', '修正配当修正方向', '予想売上高', '予想営業益', '予想経常益', '予想最終益', '予想修正配当'],
      dtype='object')

In [319]:
# 不要行、不要列の削除、並び替え
# 実績(と修正配当)はいる?いらない?
# 実績の発表と同時に次の予想が出ているのでやっぱりここではいらないのかな?
tables[12] = tables[12].ix[tables[12].index % 2 == 0, ['会計基準', '決算期', '予想売上高', '予想営業益', '予想経常益', '予想最終益', '発表日']].reset_index(drop=True)
tables[12] = tables[12].ix[tables[12]['決算期'] != '実績']

In [320]:
# 決算期の NaN 埋め
tables[12]['決算期'] = tables[12]['決算期'].fillna(method='ffill')

In [321]:
# 100万円単位換算
tables[12][['予想売上高', '予想営業益', '予想経常益', '予想最終益']] = tables[12][['予想売上高', '予想営業益', '予想経常益', '予想最終益']].apply(lambda x: x * 1000000)

In [322]:
# 日付のパース、datetime.dateへの型変換
# tables[12]['決算期'] = tables[12]['決算期'].apply(lambda x: datetime.strptime(x, '%Y.%m').date()) # 日付ではないので文字列のままの方がいいかも？
tables[12]['発表日'] = tables[12]['発表日'].apply(lambda x: parse(x, yearfirst=True).date())
# pandasのTimestampへの型変換
tables[12]['発表日'] = pd.to_datetime(tables[12]['発表日'], format='%Y-%m-%d')
# tables[12]['決算期'] = pd.to_datetime(tables[12]['決算期'], format='%Y-%m-%d')

In [104]:
# 修正配当用の処理なので不要
# '－'  を NaN に置換
# tables[12].loc[~tables[12]['修正配当'].str.isnumeric(), '修正配当'] = np.nan
# 型変換
# tables[12]['修正配当'] = tables[12]['修正配当'].astype(float)

In [323]:
tables[12]

,会計基準,決算期,予想売上高,予想営業益,予想経常益,予想最終益,発表日
0,U,2013.03,2.200000e+13,1.000000e+12,1.160000e+12,7.600000e+11,2012-05-09
1,U,2013.03,2.130000e+13,1.050000e+12,1.180000e+12,7.800000e+11,2012-11-05
2,U,2013.03,2.180000e+13,1.150000e+12,1.290000e+12,8.600000e+11,2013-02-05
4,U,2014.03,2.350000e+13,1.800000e+12,1.890000e+12,1.370000e+12,2013-05-08
5,U,2014.03,2.400000e+13,1.940000e+12,2.030000e+12,1.480000e+12,2013-08-02
6,U,2014.03,2.500000e+13,2.200000e+12,2.290000e+12,1.670000e+12,2013-11-06
7,U,2014.03,2.550000e+13,2.400000e+12,2.530000e+12,1.900000e+12,2014-02-04
9,U,2015.03,2.570000e+13,2.300000e+12,2.390000e+12,1.780000e+12,2014-05-08
10,U,2015.03,2.650000e+13,2.500000e+12,2.700000e+12,2.000000e+12,2014-11-05
11,U,2015.03,2.700000e+13,2.700000e+12,2.920000e+12,2.130000e+12,2015-02-04


In [255]:
tables[12].dtypes

会計基準            object
決算期             object
売上高            float64
営業益            float64
経常益            float64
最終益            float64
修正日     datetime64[ns]
dtype: object

## tables[23] 過去最高 【実績】

In [33]:
tables[23]

,Unnamed: 0,売上高,営業益,経常益,最終益,１株益
0,過去最高,28403118.00,2853971.00,2983381.00,2312694.00,741.40
1,決算期,2016.03,2016.03,2016.03,2016.03,2016.03


## tables[24] 下期業績 (過去3年 + 今年予想 + 前年同期比)

In [34]:
tables[24]

,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,U 14.10-03,14288924.0,1398618.0,1383654.0,1046502.0,331.3,125,15/05/08
1,U 15.10-03,14311630.0,1270566.0,1308230.0,1054582.0,338.1,110,16/05/11
2,U 16.10-03,14526660.0,877507.0,1017291.0,884936.0,292.6,110,17/05/10
3,U 予 17.10-03,14308793.0,903459.0,997827.0,878672.0,298.6,－,17/11/07
4,前年同期比,-1.5,3.0,-1.9,-0.7,2.1,NaN,(%)


In [50]:
tables[24].columns

Index(['決算期', '売上高', '営業益', '経常益', '最終益', '１株益', '１株配', '発表日'], dtype='object')

## tables[25] 第２四半期累計決算【実績】 (過去3年 + 前年同期比)

In [35]:
tables[25]

,決算期,売上高,営業益,経常益,最終益,１株益,対通期進捗率,発表日
0,U 15.04-09,14091488.0,1583405.0,1675151.0,1258112.0,399.4,56.1,15/11/05
1,U 16.04-09,13070533.0,1116865.0,1176534.0,946173.0,311.1,53.6,16/11/08
2,U 17.04-09,14191207.0,1096541.0,1252173.0,1071328.0,359.6,55.7,17/11/07
3,前年同期比,8.6,-1.8,6.4,13.2,15.6,NaN,(%)


In [278]:
kabupro.ix[(kabupro['証券コード'] == code) & (kabupro['会計基準'] == '米国基準') & (kabupro['決算期間'] == '第2四半期'), 
           ['連結個別', '期首', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益', '情報公開日 (更新日)']].tail(3)

,連結個別,期首,売上高,営業利益,経常利益,純利益,一株当り純利益,情報公開日 (更新日)
70501,連結,2015-04-01,1.409149e+13,1.583405e+12,1.675151e+12,1.258112e+12,399.39,2016-11-08
70505,連結,2016-04-01,1.307053e+13,1.116865e+12,1.176534e+12,9.461730e+11,311.08,2017-11-07
70509,連結,2017-04-01,1.419121e+13,1.096541e+12,1.252173e+12,1.071328e+12,359.55,2017-11-07


In [294]:
kabupro.ix[(kabupro['証券コード'] == code)& (kabupro['会計基準'] == '米国基準') & (kabupro['決算期間'].isin(['第2四半期', '通期'])), # 
           ['連結個別', '期首', '決算期間', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益', '情報公開日 (更新日)']].tail(5)

,連結個別,期首,決算期間,売上高,営業利益,経常利益,純利益,一株当り純利益,情報公開日 (更新日)
70501,連結,2015-04-01,第2四半期,1.409149e+13,1.583405e+12,1.675151e+12,1.258112e+12,399.39,2016-11-08
70503,連結,2015-04-01,通期,2.840312e+13,2.853971e+12,2.983381e+12,2.312694e+12,741.36,2017-05-10
70505,連結,2016-04-01,第2四半期,1.307053e+13,1.116865e+12,1.176534e+12,9.461730e+11,311.08,2017-11-07
70507,連結,2016-04-01,通期,2.759719e+13,1.994372e+12,2.193825e+12,1.831109e+12,605.47,2017-05-10
70509,連結,2017-04-01,第2四半期,1.419121e+13,1.096541e+12,1.252173e+12,1.071328e+12,359.55,2017-11-07


In [295]:
tables[12].tail(1)

,会計基準,決算期,売上高,営業益,経常益,最終益,修正日
25,U,2018.03,2.850000e+13,2.000000e+12,2.250000e+12,1.950000e+12,2017-11-07


In [42]:
# 比較参照用
kabupro.columns

Index(['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目 (売上高欄)', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益',
       '希薄化後一株当り純利益', '純資産又は株主資本', '総資産', '一株当り純資産', '営業キャッシュフロー',
       '投資キャッシュフロー', '財務キャッシュフロー', '情報公開日 (更新日)'],
      dtype='object')

## tables[26] ３ヵ月業績の推移【実績】(過去5年 + 前年同期比) 累積ではなく差分

In [368]:
# 保存した html からテーブル属性を読み込み
tables = pd.read_html('/Users/Really/Stockyard/_kabutan_html/kabutan_{0}.html'.format(code), header=0)

In [334]:
tables[26]
# 株プロに無い項目: 売上営業損益率 = 営業益 / 売上高?

,会計基準,決算期,四半期売上高,四半期営業益,四半期経常益,四半期最終益,四半期１株益,四半期売上営業損益率,発表日
10,U,2015-04-06,6.987648e+12,7.560010e+11,8.452590e+11,6.463940e+11,205.4,10.8,2015-08-04
11,U,2015-07-09,7.103840e+12,8.274040e+11,8.298920e+11,6.117180e+11,194.2,11.6,2015-11-05
12,U,2015-10-12,7.339882e+12,7.222660e+11,7.777610e+11,6.279650e+11,200.2,9.8,2016-02-05
13,U,2016-01-03,6.971748e+12,5.483000e+11,5.304690e+11,4.266170e+11,136.8,7.9,2016-05-11
14,U,2016-04-06,6.589113e+12,6.422300e+11,6.770560e+11,5.524650e+11,181.1,9.7,2016-08-04
15,U,2016-07-09,6.481420e+12,4.746350e+11,4.994780e+11,3.937080e+11,129.4,7.3,2016-11-08
16,U,2016-10-12,7.084187e+12,4.385860e+11,5.875380e+11,4.865310e+11,160.4,6.2,2017-02-06
17,U,2017-01-03,7.442473e+12,4.389210e+11,4.297530e+11,3.984050e+11,131.7,5.9,2017-05-10
18,U,2017-04-06,7.047606e+12,5.742940e+11,6.793480e+11,6.130560e+11,205.1,8.1,2017-08-04
19,U,2017-07-09,7.143601e+12,5.222470e+11,5.728250e+11,4.582720e+11,153.8,7.3,2017-11-07


In [369]:
# 全ての列項目がnullの行を除去
tables[26] = tables[26][~tables[26].isnull().all(axis=1)].reset_index(drop=True)

In [370]:
# 前年同期比の行を除去
tables[26] = tables[26][~tables[26]['決算期'].str.contains('前年同期比')].reset_index(drop=True)

In [371]:
# 決算期列の要素を会計基準と決算期に分割、それぞれの列に代入(同時に会計基準列を新規作成)
tables[26][['会計基準', '四半期期首']] = pd.DataFrame(list(tables[26]['決算期'].str.split(' ')))

In [372]:
# 列の並び替え
tables[26] = tables[26][['会計基準', '四半期期首', '売上高', '営業益', '経常益', '最終益', '１株益', '売上営業損益率', '発表日']]

In [373]:
tables[26].columns

Index(['会計基準', '四半期期首', '売上高', '営業益', '経常益', '最終益', '１株益', '売上営業損益率', '発表日'], dtype='object')

In [374]:
tables[26].columns = ['会計基準', '四半期期首', '四半期売上高', '四半期営業益', '四半期経常益', '四半期最終益', '四半期１株益', '四半期売上営業損益率', '発表日']

In [375]:
# 100万円単位換算
tables[26][['四半期売上高', '四半期営業益', '四半期経常益', '四半期最終益']] = tables[26][['四半期売上高', '四半期営業益', '四半期経常益', '四半期最終益']].apply(lambda x: x * 1000000)

In [376]:
# 日付のパース、datetime.dateへの型変換
tables[26]['四半期期首'] = tables[26]['四半期期首'].apply(lambda x: parse(x.replace('-', '.'), yearfirst=True).date())
tables[26]['発表日'] = tables[26]['発表日'].apply(lambda x: parse(x, yearfirst=True).date())
# pandasのTimestampへの型変換
tables[26]['四半期期首'] = pd.to_datetime(tables[26]['四半期期首'], format='%Y-%m-%d')
tables[26]['発表日'] = pd.to_datetime(tables[26]['発表日'], format='%Y-%m-%d')

In [377]:
tables[26].dtypes

会計基準                  object
四半期期首         datetime64[ns]
四半期売上高               float64
四半期営業益               float64
四半期経常益               float64
四半期最終益               float64
四半期１株益               float64
四半期売上営業損益率           float64
発表日           datetime64[ns]
dtype: object

__比較検証用に株プロの四半期業績の差分を作ってみる__

__もうちょっと上手いやり方ありそう__

In [ ]:
kabupro.columns

In [ ]:
kabupro.ix[(kabupro['証券コード'] == code) & (kabupro['会計基準'] == '米国基準'), 
           ['決算期', '期末', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益', '情報公開日 (更新日)']].tail(10)

In [355]:
diff_test = kabupro.ix[(kabupro['証券コード'] == code) & (kabupro['会計基準'] == '米国基準'), 
           ['決算期', '期末', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益', '情報公開日 (更新日)']]

In [358]:
diff_test

,決算期,期末,売上高,営業利益,経常利益,純利益,一株当り純利益,情報公開日 (更新日),売上高差分,営業利益差分,経常利益差分,純利益差分,一株当り純利益差分
70484,2013年3月期,2012-06-30,5.501573e+12,3.531430e+11,4.152030e+11,2.903470e+11,91.68,2013-08-02,5.501573e+12,3.531430e+11,4.152030e+11,2.903470e+11,91.68
70485,2013年3月期,2012-09-30,1.090835e+13,6.937500e+11,7.945370e+11,5.482690e+11,173.13,2013-11-06,1.090835e+13,6.937500e+11,7.945370e+11,5.482690e+11,173.13
70486,2013年3月期,2012-12-31,1.622711e+13,8.185070e+11,9.257860e+11,6.481830e+11,204.68,2014-02-04,1.622711e+13,8.185070e+11,9.257860e+11,6.481830e+11,204.68
70487,2013年3月期,2013-03-31,2.206419e+13,1.320888e+12,1.403649e+12,9.621630e+11,303.82,2014-05-08,2.206419e+13,1.320888e+12,1.403649e+12,9.621630e+11,303.82
70488,2014年3月期,2013-06-30,6.255319e+12,6.633830e+11,7.241630e+11,5.621940e+11,177.45,2014-08-05,6.255319e+12,6.633830e+11,7.241630e+11,5.621940e+11,177.45
70489,2014年3月期,2013-09-30,1.253748e+13,1.255475e+12,1.343525e+12,1.000623e+12,315.80,2014-11-05,1.253748e+13,1.255475e+12,1.343525e+12,1.000623e+12,315.80
70490,2014年3月期,2013-12-31,1.912253e+13,1.855984e+12,2.022481e+12,1.526087e+12,481.60,2015-02-04,1.912253e+13,1.855984e+12,2.022481e+12,1.526087e+12,481.60
70491,2014年3月期,2014-03-31,2.569191e+13,2.292112e+12,2.441080e+12,1.823119e+12,575.30,2015-05-08,2.569191e+13,2.292112e+12,2.441080e+12,1.823119e+12,575.30
70492,2015年3月期,2014-06-30,6.390688e+12,6.927280e+11,7.718260e+11,5.877740e+11,185.43,2014-08-05,6.390688e+12,6.927280e+11,7.718260e+11,5.877740e+11,185.43
70493,2015年3月期,2014-06-30,6.390688e+12,6.927280e+11,7.718260e+11,5.877740e+11,185.43,2015-08-04,6.390688e+12,6.927280e+11,7.718260e+11,5.877740e+11,185.43


In [356]:
diff_test[['売上高差分', '営業利益差分', '経常利益差分', '純利益差分', '一株当り純利益差分']] = diff_test[['売上高', '営業利益', '経常利益', '純利益', '一株当り純利益']]

In [362]:
diff_test.index[1]

70485

In [363]:
for count in range(diff_test.index[1], diff_test.index[1] + len(diff_test) - 1):
    if diff_test.loc[count, '決算期'] == diff_test.loc[count - 1, '決算期']:
        diff_test.loc[count, '売上高差分'] = diff_test.loc[count, '売上高'] - diff_test.loc[count - 1, '売上高']
        diff_test.loc[count, '営業利益差分'] = diff_test.loc[count, '営業利益'] - diff_test.loc[count - 1, '営業利益']
        diff_test.loc[count, '経常利益差分'] = diff_test.loc[count, '経常利益'] - diff_test.loc[count - 1, '経常利益']
        diff_test.loc[count, '純利益差分'] = diff_test.loc[count, '純利益'] - diff_test.loc[count - 1, '純利益']
        diff_test.loc[count, '一株当り純利益差分'] = diff_test.loc[count, '一株当り純利益'] - diff_test.loc[count - 1, '一株当り純利益']

In [364]:
diff_test[['決算期', '期末', '売上高差分', '営業利益差分', '経常利益差分', '純利益差分', '一株当り純利益差分', '情報公開日 (更新日)']]
# 一株当り純利益差分が株探の１株益と揃わない

,決算期,期末,売上高差分,営業利益差分,経常利益差分,純利益差分,一株当り純利益差分,情報公開日 (更新日)
70484,2013年3月期,2012-06-30,5.501573e+12,3.531430e+11,4.152030e+11,2.903470e+11,91.68,2013-08-02
70485,2013年3月期,2012-09-30,5.406781e+12,3.406070e+11,3.793340e+11,2.579220e+11,81.45,2013-11-06
70486,2013年3月期,2012-12-31,5.318752e+12,1.247570e+11,1.312490e+11,9.991400e+10,31.55,2014-02-04
70487,2013年3月期,2013-03-31,5.837086e+12,5.023810e+11,4.778630e+11,3.139800e+11,99.14,2014-05-08
70488,2014年3月期,2013-06-30,6.255319e+12,6.633830e+11,7.241630e+11,5.621940e+11,177.45,2014-08-05
70489,2014年3月期,2013-09-30,6.282166e+12,5.920920e+11,6.193620e+11,4.384290e+11,138.35,2014-11-05
70490,2014年3月期,2013-12-31,6.585044e+12,6.005090e+11,6.789560e+11,5.254640e+11,165.80,2015-02-04
70491,2014年3月期,2014-03-31,6.569382e+12,4.361280e+11,4.185990e+11,2.970320e+11,93.70,2015-05-08
70492,2015年3月期,2014-06-30,6.390688e+12,6.927280e+11,7.718260e+11,5.877740e+11,185.43,2014-08-05
70493,2015年3月期,2014-06-30,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,2015-08-04


In [378]:
tables[26]

,会計基準,四半期期首,四半期売上高,四半期営業益,四半期経常益,四半期最終益,四半期１株益,四半期売上営業損益率,発表日
0,U,2012-10-12,5.318752e+12,1.247570e+11,1.312490e+11,9.991400e+10,31.6,2.3,2013-02-05
1,U,2013-01-03,5.837086e+12,5.023810e+11,4.778630e+11,3.139800e+11,99.1,8.6,2013-05-08
2,U,2013-04-06,6.255319e+12,6.633830e+11,7.241630e+11,5.621940e+11,177.4,10.6,2013-08-02
3,U,2013-07-09,6.282166e+12,5.920920e+11,6.193620e+11,4.384290e+11,138.4,9.4,2013-11-06
4,U,2013-10-12,6.585044e+12,6.005090e+11,6.789560e+11,5.254640e+11,165.8,9.1,2014-02-04
5,U,2014-01-03,6.569382e+12,4.361280e+11,4.185990e+11,2.970320e+11,93.7,6.6,2014-05-08
6,U,2014-04-06,6.390688e+12,6.927280e+11,7.718260e+11,5.877740e+11,185.4,10.8,2014-08-05
7,U,2014-07-09,6.554909e+12,6.592180e+11,7.373480e+11,5.390620e+11,170.3,10.1,2014-11-05
8,U,2014-10-12,7.170017e+12,7.628800e+11,8.464610e+11,6.000270e+11,189.7,10.6,2015-02-04
9,U,2015-01-03,7.118907e+12,6.357380e+11,5.371930e+11,4.464750e+11,141.3,8.9,2015-05-08


## tables[27] 財務 【実績】

In [379]:
# 保存した html からテーブル属性を読み込み
tables = pd.read_html('/Users/Really/Stockyard/_kabutan_html/kabutan_{0}.html'.format(code), header=0)

In [388]:
tables[27]
# 株プロに無い項目: 自己資本比率, 自己資本, 剰余金, 有利子負債倍率

,決算期,１株純資産,自己資本比率,総資産,自己資本,剰余金,有利子負債倍率,発表日
0,連 1998.03,1583.20,43.5,13854355.0,6021895.0,5213340.0,－,1998-08-01
1,連 1999.03,1642.90,41.9,14753311.0,6175937.0,5367879.0,－,1999-08-01
2,連 2000.03,1841.50,41.3,16469054.0,6796666.0,6014125.0,0.64,2000-05-17
3,連 2001.03,1956.57,40.6,17519427.0,7114567.0,6162656.0,0.63,2001-05-16
4,連 2002.03,2059.94,36.8,19888937.0,7325072.0,6527956.0,0.79,2002-05-13
5,U 2003.03,2063.43,35.3,20152974.0,7121000.0,7219896.0,1.02,2003-05-08
6,U 2004.03,2456.08,37.1,22040228.0,8178567.0,8326215.0,0.92,2004-05-11
7,U 2005.03,2767.67,37.2,24335011.0,9044950.0,9332176.0,0.95,2005-05-10
8,U 2006.03,3257.63,36.8,28731595.0,10560449.0,10459788.0,0.98,2006-05-10
9,U 2007.03,3701.17,36.3,32574779.0,11836092.0,11764713.0,1.02,2007-05-09


In [208]:
tables[27].columns

Index(['決算期', '１株純資産', '自己資本比率', '総資産', '自己資本', '剰余金', '有利子負債倍率', '発表日',
       '会計基準'],
      dtype='object')

In [383]:
# 7203 トヨタ 元の値が '90/08/01' で明らかにおかしい。これどうしよう。。。
tables[27].ix[2, '発表日'] = '99/08/01'

In [386]:
# 全ての列項目がnullの行を除去
tables[27] = tables[27][~tables[27].isnull().all(axis=1)].reset_index(drop=True)

In [308]:
# 決算期列の要素を会計基準と決算期に分割、それぞれの列に代入(同時に会計基準列を新規作成)
tables[27][['会計基準', '決算期']] = pd.DataFrame(list(tables[27]['決算期'].str.split(' ')))

In [309]:
# 列の並び替え
tables[27] = tables[27][['会計基準', '決算期', '１株純資産', '自己資本比率', '総資産', '自己資本', '剰余金', '有利子負債倍率', '発表日']]

In [310]:
# 決算期が 'yyyy.mm' 表記ではない行は確定決算前と判断して削除
tables[27] = tables[27][tables[27]['決算期'].str.contains('\d\d\d\d.\d\d')]

In [311]:
# '－'  を NaN に置換
# .str を2回も使わないといけないのはなんだか。。。
tables[27].loc[~tables[27]['１株純資産'].str.replace('.', '').str.isnumeric(), '１株純資産'] = np.nan
tables[27].loc[~tables[27]['有利子負債倍率'].str.replace('.', '').str.isnumeric(), '有利子負債倍率'] = np.nan

In [312]:
# 型変換
tables[27][['１株純資産', '有利子負債倍率']] = tables[27][['１株純資産', '有利子負債倍率']].astype(float)

In [313]:
# 100万円単位換算
tables[27][['総資産', '自己資本', '剰余金']] = tables[27][['総資産', '自己資本', '剰余金']].apply(lambda x: x * 1000000)

In [387]:
# 日付のパース、datetime.dateへの型変換
# tables[27]['決算期'] = tables[27]['決算期'].apply(lambda x: parse(x.replace('-', '.'), yearfirst=True).date()) # 日付ではないので文字列のままの方がいいかも？
tables[27]['発表日'] = tables[27]['発表日'].apply(lambda x: parse(x, yearfirst=True).date())
# pandasのTimestampへの型変換
# tables[27]['決算期'] = pd.to_datetime(tables[27]['決算期'], format='%Y-%m-%d')
tables[27]['発表日'] = pd.to_datetime(tables[27]['発表日'], format='%Y-%m-%d')

In [241]:
tables[27].dtypes

会計基準               object
決算期                object
１株純資産             float64
自己資本比率            float64
総資産               float64
自己資本              float64
剰余金               float64
有利子負債倍率           float64
発表日        datetime64[ns]
dtype: object

In [242]:
tables[27][15:]
# 株プロに無い項目: 自己資本比率, 自己資本, 剰余金, 有利子負債倍率

,会計基準,決算期,１株純資産,自己資本比率,総資産,自己資本,剰余金,有利子負債倍率,発表日
15,U,2013.03,3835.30,34.2,3.548332e+13,1.214804e+13,1.268921e+13,1.16,2013-05-08
16,U,2014.03,4564.74,34.9,4.143747e+13,1.446915e+13,1.411630e+13,1.13,2014-05-08
17,U,2015.03,5334.96,35.2,4.772983e+13,1.678813e+13,1.559195e+13,1.13,2015-05-08
18,U,2016.03,5513.08,35.3,4.742760e+13,1.674694e+13,1.679424e+13,1.09,2016-05-11
19,U,2017.03,5887.88,35.9,4.875019e+13,1.751481e+13,1.760107e+13,1.09,2017-05-10


In [74]:
kabupro.ix[(kabupro['証券コード'] == code) & (kabupro['会計基準'] == '米国基準') & (kabupro['決算期間'] == '通期'), 
           ['連結個別', '決算期', '期首', '一株当り純資産', '総資産', '情報公開日 (更新日)']]

,連結個別,決算期,期首,一株当り純資産,総資産,情報公開日 (更新日)
70487,連結,2013年3月期,2012-04-01,3835.30,3.548332e+13,2014-05-08
70491,連結,2014年3月期,2013-04-01,4564.74,4.143747e+13,2015-05-08
70498,連結,2015年3月期,2014-04-01,5334.96,4.772983e+13,2015-05-08
70499,連結,2015年3月期,2014-04-01,5334.96,4.772983e+13,2016-05-11
70503,連結,2016年3月期,2015-04-01,5513.08,4.742760e+13,2017-05-10
70507,連結,2017年3月期,2016-04-01,5887.88,4.875019e+13,2017-05-10


In [42]:
# 比較参照用
kabupro.columns

Index(['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目 (売上高欄)', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益',
       '希薄化後一株当り純利益', '純資産又は株主資本', '総資産', '一株当り純資産', '営業キャッシュフロー',
       '投資キャッシュフロー', '財務キャッシュフロー', '情報公開日 (更新日)'],
      dtype='object')

## おしまい

In [38]:
tables[29]

IndexError: list index out of range

# (準備) 単一銘柄の決算ページの取得

In [9]:
# 個別銘柄の決算ページを開く
code = 9437 # トヨタ

print('Navigating...', file=sys.stderr)
browser.open('https://kabutan.jp/stock/finance?code={0}&mode=k'.format(code))

# 決算ページにいることを確認する
assert '決算' in browser.parsed.title.string

print(browser.select('.kobetsu_data_table1_meigara')[0].text.strip())

Navigating...


ＮＴＴドコモ


## html 全体の取得と保存

In [10]:
kessan_html = browser.find()
kessan_html

<html lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<title>ＮＴＴドコモ (ＮＴＴドコモ) 【9437】：決算/業績・財務推移 [通期/半期/四半期] | 株探</title>
<meta content="ＮＴＴドコモ の直近5期の業績推移表、独自の8四半期(3ヵ月毎)の業績推移表、3期の財務データ推移表など豊富な決算情報が満載！ 経常利益の進捗率、採算性がわかる売上営業利益率、有利子負債倍率などプロ仕様のデータで、アナリスト並みの業績・財務分析ができます。ＮＴＴドコモ による決算発表・業績修正はリアルタイムで更新。業績推移表は、売上高 営業利益 経常利益 最終利益(純利益) 1株利益 1株配当 決算発表日 を表記。財務データは 1株純資産 自己資本比率 総資産 純資産 剰余金 有利子負債倍率 を表記。四半期累計や半期(上期・下期)の業績推移表も掲載しています。" name="description"/>
<meta content="ＮＴＴドコモ,えぬてぃてぃどこも,えぬてぃーてぃーどこも,NTT DOCOMO INC.,9437,情報・通信業,株式,株価,投資,決算,業績推移,決算発表,業績修正,通期,四半期,3ヵ月,四半期累計,上期,上半期,下期,下半期,進捗率,進捗度,売上営業利益率,売上高,営業利益,経常利益,最終利益,純利益,1株利益,1株配当,決算発表日,1株純資産,自己資本比率,株主資本比率,総資産,純資産,自己資本,株主資本,剰余金,有利子負債倍率" name="keywords"/>
<meta content="9TyMQp63vRE7zxgZJofMB6CJlyYQBwnVoHJoVFo1_rg" name="google-site-verificatio

In [12]:
tables = pd.read_html(str(kessan_html), header=0)
tables[11]

,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,連 1999.03,3118398.0,508548.0,350346.0,204815.0,118224.0,5000.0,99/05/25
2,連 2000.03,3718694.0,545760.0,503116.0,252139.0,26330.0,1000.0,00/05/24
3,連 2001.03,4686004.0,777162.0,686918.0,365505.0,37984.0,1000.0,01/05/09
4,U 2002.03,4659254.0,1000887.0,956391.0,-116191.0,-11577.0,1500.0,02/05/08
5,U 2003.03,4809088.0,1056719.0,1042968.0,212491.0,4253.8,500.0,03/05/08
6,U 2004.03,5048065.0,1102918.0,1101123.0,650007.0,13099.0,1500.0,04/05/07
7,U 2005.03,4844610.0,784166.0,1288221.0,747564.0,15771.0,2000.0,05/05/10
8,U 2006.03,4765872.0,832639.0,952303.0,610481.0,13491.0,4000.0,06/04/28
9,U 2007.03,4788093.0,773524.0,772943.0,457278.0,10396.0,4000.0,07/04/27


In [188]:
# 後でhtml形式で読み込み可能なファイルとして書き出す方法
kabutan_kessan = open('kabutan_kessan.html', 'w')
kabutan_kessan.write(str(browser.find()))
kabutan_kessan.close()

In [191]:
# 保存したhtmlファイルからの読み込み
tables = pd.read_html('/Users/Really/Stockyard/kabutan_kessan.html', header=0)

## html 内のテーブル属性のみの取得と保存

In [133]:
tables = pd.read_html(str(browser.select('table')), header=0)
tables[11]

Navigating...


,決算期,修正日,修正方向,売上高,営業益,経常益,最終益,修正配当,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,U,2013.03,12/05/09,NaN,NaN,NaN,NaN,NaN,NaN,22000000.0,1000000.0,1160000.0,760000.0,－
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,U,NaN,12/11/05,↓↑↑↑‐,↓,↑,↑,↑,‐,21300000.0,1050000.0,1180000.0,780000.0,－
3,↓,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,U,NaN,13/02/05,↑↑↑↑‐,↑,↑,↑,↑,‐,21800000.0,1150000.0,1290000.0,860000.0,－
5,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,U,実績,13/05/08,↑↑↑↑‐,↑,↑,↑,↑,‐,22064192.0,1320888.0,1403649.0,962163.0,90
7,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,U,2014.03,13/05/08,NaN,NaN,NaN,NaN,NaN,NaN,23500000.0,1800000.0,1890000.0,1370000.0,－
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [190]:
# 後でhtml形式で読み込み可能なファイルとして書き出す方法
kabutan_kessan_tables = open('kabutan_kessan_tables.html', 'w')
kabutan_kessan_tables.write(str(browser.select('table')))
kabutan_kessan_tables.close()

In [109]:
# 保存したhtmlファイルからの読み込み
tables = pd.read_html('/Users/Really/Stockyard/kabutan_kessan_tables.html', header=0)

# (準備) テーブル内容の確認 (未ログイン)

In [6]:
url = 'https://kabutan.jp/stock/finance?code=1301&mode=k#kakosaikou_zisseki'

tables = get_table(url)

In [8]:
len(tables)

22

In [31]:
# ページ上部の主要指標情報 1-4の各項目と同じところっぽい
tables[0]

,"日経平均 (11:30) 22,404.85 +53.73 0.24%",日経平均 (11:30),"22,404.85",+53.73,0.24%,米ドル円 (11:58) 112.40 -0.84 -0.74%,米ドル円 (11:58),112.40,-0.84,-0.74%,"ＮＹダウ (終値) 23,458.36 +187.08 0.80%",ＮＹダウ (終値),"23,458.36",+187.08,0.80%,"上海総合 (終値) 3,399.87 -0.79 -0.02%",上海総合 (終値),"3,399.87",-0.79,-0.02%
0,日経平均 (11:30),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22404.85,53.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.24%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,米ドル円 (11:58),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,112.40,-0.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-0.74%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ＮＹダウ (終値),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,23458.36,187.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.80%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,上海総合 (終値),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# 主要指標情報 日経平均
tables[1]

,日経平均 (11:30),Unnamed: 1
0,22404.85,53.73
1,0.24%,NaN


In [10]:
# 主要指標情報 米ドル円
tables[2]

,米ドル円 (11:58),Unnamed: 1
0,112.40,-0.84
1,-0.74%,NaN


In [11]:
# 主要指標情報 ＮＹダウ (終値)
tables[3]

,ＮＹダウ (終値),Unnamed: 1
0,23458.36,187.08
1,0.80%,NaN


In [12]:
# 主要指標情報 上海総合 (終値)
tables[4]

,上海総合 (終値),Unnamed: 1
0,3399.87,-0.79
1,-0.02%,NaN


In [13]:
# 検索窓
tables[5]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,「株探」で配信したニュース記事を検索します,Unnamed: 5


In [14]:
# 銘柄概要
tables[6]

,1301 極洋 東証１ 11:29,1301,極洋,東証１,11:29,Unnamed: 5
0,1301,極洋,東証１,11:29,NaN,NaN
1,"3,645円 前日比 -30 (-0.82%) 比較される銘柄： マルハニチロ 、日水 、カネコ種","3,645円",前日比,-30,(-0.82%),比較される銘柄： マルハニチロ 、日水 、カネコ種
2,"3,645円",前日比,-30,(-0.82%),NaN,NaN
3,比較される銘柄： マルハニチロ 、日水 、カネコ種,NaN,NaN,NaN,NaN,NaN


In [15]:
# 銘柄概要
tables[7]

,1301,極洋,東証１,11:29


In [16]:
# 銘柄概要
tables[8]

,"3,645円",前日比,-30,(-0.82%)
0,比較される銘柄： マルハニチロ 、日水 、カネコ種,NaN,NaN,NaN


In [17]:
# 銘柄概要
tables[9]

,業績：
0,水産・農林業
1,単位 100株


In [18]:
# ＰＥＲ ＰＢＲ 利回り 信用倍率
tables[10]

,ＰＥＲ,ＰＢＲ,利回り,信用倍率
0,14.2倍,1.46倍,1.37％,1.80倍


In [19]:
# 通期業績
tables[11]
# 株プロに無い項目: １株配

,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014.03,202387.0,2915.0,2985.0,2968.0,28.3,5.0,14/05/09
2,2015.03,218350.0,2460.0,2107.0,2433.0,23.2,5.0,15/05/08
3,2016.03,226626.0,2433.0,2814.0,1799.0,17.1,5.0,16/05/09
4,2017.03,236561.0,3723.0,3709.0,2422.0,230.7,60.0,17/05/11
5,予 2018.03,250000.0,4000.0,4000.0,2700.0,257.0,50.0,17/05/11
6,前期比,5.7,7.4,7.8,11.5,11.4,NaN,(%)
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# 業績予想修正履歴
tables[12]

,決算期,修正日,修正方向,売上高,営業益,経常益,最終益,修正配当,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,2013.03,12/05/11,NaN,NaN,NaN,NaN,NaN,NaN,185000.0,3200.0,3100.0,1800.0,50.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,12/10/26,↓↓↓↓→,↓,↓,↓,↓,→,175000.0,2000.0,2200.0,1100.0,50.0
3,↓,↓,↓,↓,→,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,実績,13/05/10,↑↑↑↑→,↑,↑,↑,↑,→,178046.0,2324.0,2262.0,1269.0,50.0
5,↑,↑,↑,↑,→,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,2014.03,13/05/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,13/10/21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,実績,14/05/09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,2015.03,14/05/09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# 不明
tables[13]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4


In [22]:
# 修正履歴修正方向
tables[14]

,↓,↓.1,↓.2,↓.3,→


In [23]:
# 修正履歴修正方向
tables[15]

,↑,↑.1,↑.2,↑.3,→


In [24]:
# 過去最高 【実績】
tables[16]

,Unnamed: 0,売上高,営業益,経常益,最終益,１株益
0,過去最高,236561.00,3723.00,3709.00,5540.0,230.70
1,決算期,2017.03,2017.03,2017.03,1985.1,2017.03


In [25]:
# 下期業績
tables[17]

,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,14.10-03,115874.0,1739.0,1421.0,168.0,1.6,5.0,15/05/08
1,15.10-03,118127.0,1307.0,1407.0,888.0,8.5,5.0,16/05/09
2,16.10-03,126991.0,2552.0,2705.0,1236.0,117.7,60.0,17/05/11
3,予 17.10-03,129542.0,1754.0,1604.0,1067.0,101.6,50.0,17/11/06
4,前年同期比,2.0,-31.3,-40.7,-13.7,-13.7,NaN,(%)


In [26]:
# 第２四半期累計決算【実績】
tables[18]

,決算期,売上高,営業益,経常益,最終益,１株益,対通期進捗率,発表日
0,15.04-09,108499.0,1126.0,1407,911.0,8.7,50.0,15/11/06
1,16.04-09,109570.0,1171.0,1004,1186.0,113.0,27.1,16/11/04
2,17.04-09,120458.0,2246.0,2396,1633.0,155.5,59.9,17/11/06
3,前年同期比,9.9,91.8,2.4倍,37.7,37.6,NaN,(%)


In [27]:
# ３ヵ月業績の推移【実績】
# 累積ではなく差分？
tables[19]

,決算期,売上高,営業益,経常益,最終益,１株益,売上営業損益率,発表日
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15.10-12,70391.0,1341.0,1281,222.0,2.1,1.9,16/02/05
2,16.01-03,47736.0,-34.0,126,666.0,6.3,-0.1,16/05/09
3,16.04-06,52206.0,467.0,380,551.0,5.2,0.9,16/08/05
4,16.07-09,57364.0,704.0,624,635.0,60.5,1.2,16/11/04
5,16.10-12,70405.0,1701.0,1823,1263.0,120.3,2.4,17/02/10
6,17.01-03,56586.0,851.0,882,-27.0,-2.6,1.5,17/05/11
7,17.04-06,56844.0,979.0,1103,754.0,71.8,1.7,17/08/04
8,17.07-09,63614.0,1267.0,1293,879.0,83.7,2.0,17/11/06
9,前年同期比,10.9,80.0,2.1倍,38.4,38.3,NaN,(%)


In [28]:
# 財務 【実績】
tables[20]
# 株プロに無い項目: 自己資本比率, 自己資本, 剰余金, 有利子負債倍率

,決算期,１株純資産,自己資本比率,総資産,自己資本,剰余金,有利子負債倍率,発表日
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015.03,215.65,25.5,88937.0,22649.0,16537.0,2.09,15/05/08
2,2016.03,214.97,23.9,94608.0,22578.0,17739.0,2.24,16/05/09
3,2017.03,2378.09,25.6,97391.0,24976.0,19637.0,2.04,17/05/11
4,17.04-09,－,22.0,119806.0,26312.0,20640.0,2.50,17/11/06
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# 日経平均チャート切り替え
tables[21]

,Unnamed: 0,日中足
0,日 足,NaN
